# Sleeper Standings

Use [Sleeper API Documentation](https://docs.sleeper.com/) to complete this.

Sample League Sleeper IDs:
 1. QUESTASY Football Trial: ```865826998382112768```

TODO:
 - [x] Consider using [leg](https://api.sleeper.app/v1/state/nfl) for the "week" variable so you don't have to manually update it weekly
 - [X] Join [Rosters endpoint "owner_id"](https://api.sleeper.app/v1/league/844512859378528256/rosters) with [Users endpoint "user_id"](https://api.sleeper.app/v1/league/844512859378528256/users) to get "display_name" associated with overall stats.
 - [x] For weekly stats (where week < "leg") use the [matchups endpoint](https://docs.sleeper.com/#getting-matchups-in-a-league) and match user table's roster ID with matchup's roster ID

In [11]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import requests

##
## SETTINGS:
## List of Leagues
## Output: list_leagues (variable; list)
##

leagues = [999761008103067648,\
           1003091416626683904,\
           1003692357414719488
          ]


#
# Get the current week
#
# variable: num_week
#

url_nfl_state= 'https://api.sleeper.app/v1/state/nfl'
response = requests.get(url_nfl_state)
# It's -1 because I will usually pull rankings at the beginning of the following week.
current_week = response.json()['week']-1



#
# Join Rosters endpoint "owner_id" with Users endpoint "user_id" to get "display_name" associated with overall stats.
# Input: leagues
# Output table_teams; list_owners
#
# What we need:

# "Display Name": "username", "user_id": 1234, "roster_id": 4, "league_id":512321, "wins":5, "losses":5, ties:0 "points for": 234]

list_rosters_json = []
list_roster_data = []
list_users_data = []
list_matchups_data = []
i = 0

for league in leagues:
    #set display league id (i) to 1 and increment per league
    i+=1
    url_roster = f"https://api.sleeper.app/v1/league/{league}/rosters"
    roster_response = requests.get(url_roster)
    json_roster = roster_response.json()
    # do += instead of append so it keeps it flat
    list_rosters_json+=json_roster
    
#    leagueid, leagueid_display ownerId, rosterId,
    for team in json_roster:
        temp_list = [league, i, team['owner_id'], team['roster_id'], \
                     team['settings']['wins'],team['settings']['losses'],\
                     team['settings']['ties'], \
                     float(team['settings']['fpts']+team['settings']['fpts_decimal']*.01),\
                     float(team['settings']['fpts_against']+team['settings']['fpts_against_decimal']*.01)\
                    ]
        list_roster_data.append(temp_list)

#     get users endpoint
    url_users = f"https://api.sleeper.app/v1/league/{league}/users"
    users_response = requests.get(url_users)
    json_users = users_response.json()
    for team in json_users:
        temp_list = [team['user_id'],team['display_name']]
        list_users_data.append(temp_list)
        
# get matchup endpoint
    for week in range(1,current_week+1):
        url_matchups =f"https://api.sleeper.app/v1/league/{league}/matchups/{week}"
        matchups_response = requests.get(url_matchups)
        json_matchups = matchups_response.json()
        for matchup in json_matchups:
            list_matchups_data.append([league, matchup['roster_id'], matchup['points']])
        

df_roster = pd.DataFrame(list_roster_data, columns =['League ID', 'League', 'Owner ID', 'Roster ID', 'Wins', "Losses", "Ties", "Pts For", "Pts Against"])
# display(df_roster)

df_users = pd.DataFrame(list_users_data, columns = ['Owner ID', "Display Name"]) 
# display(df_users)

df_matchups = pd.DataFrame(list_matchups_data, columns = ["League ID", "Roster ID", "Pts For"])
df_matchups_2 = df_matchups.copy().drop("Roster ID", axis=1).groupby('League ID').describe()

#display(df_matchups_2)

df_grouped = df_matchups.groupby(['League ID', 'Roster ID']).describe()
df_grouped_2 = df_grouped.join(df_matchups_2, on="League ID",rsuffix=" League")
df_grouped_2.columns = ["_".join(col_name).rstrip('_') for col_name in df_grouped_2.columns.to_flat_index()]
df_grouped_2 = df_grouped_2.reset_index().drop(["Pts For_count", "Pts For League_count","Pts For League_min","Pts For League_25%","Pts For League_50%","Pts For League_75%","Pts For League_max"],axis = 1)
df_grouped_2 = df_grouped_2.assign(z_score = (df_grouped_2['Pts For_mean'] - df_grouped_2['Pts For League_mean'])/df_grouped_2['Pts For League_std'])
#display(df_grouped_2)

df_final = df_roster.join(df_users.set_index('Owner ID'), on='Owner ID', validate='1:1')
df_final = pd.merge(df_final, df_grouped_2, on=['League ID', 'Roster ID'])
df_final = df_final.reindex(columns=["Display Name","League", "Wins", "Losses", "Ties", "Pts For", "Pts Against", "z_score"]).sort_values(by=["Wins","z_score"],ascending=[False,False])
df_final = df_final.reset_index()
df_final.index+=1
df_final = df_final.drop("index",axis=1)
display(df_final)

,Display Name,League,Wins,Losses,Ties,Pts For,Pts Against,z_score
1,sturt,2,2,0,0,177.92,87.62,1.966466
2,jroffe52,1,2,0,0,146.66,117.14,1.761201
3,michaelzhao,3,2,0,0,136.54,110.24,1.715428
4,dcdragon,1,2,0,0,132.38,120.36,1.140153
5,Occasion,3,2,0,0,126.94,90.22,1.107345
6,AlexEJerome,3,2,0,0,126.10,100.02,1.054138
7,pavanbhat,2,2,0,0,144.64,133.04,0.814891
8,michaelq34,2,2,0,0,129.30,115.08,0.284087
9,jmiller3,2,2,0,0,125.96,122.64,0.168515
10,sareet,1,2,0,0,108.54,87.44,0.103334
